# Extra Credit

For upto an additional 15 extra credit points on the midterm exam, you can write another function that applies additional criteria on top of what the midterm already did.  To earn the addition points, you need write a function called `summarize_results(patients)` to take the output of your  `process_people()` function to apply a new business rule and summarize the results.

* Your function should take the dictionary created by `process_people()` as it's one input parameter: `patients`.
* If the PRIEST probably is less than 10%, change the `hospital` and `address` to both have a value of `'Home'` indicating patients with a lower probability of an adverse outcome can just go home rather than be admitted to a hospital.
* Summarize those results and count how many patients are now being sent to each location.  Your result should have the following format:
```
{
    "Home": 123,
    "Hospital A": 10,
    "Hospital B": 20,
    "Hospital C": 30
}
```

As always, your function needs good docstrings; but does not require any doctests in this case.

In [1]:
def priest(sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_bp, temperature, alertness, inspired_oxygen, performance_status):
    """
    Calculate the PRIEST Clinical Severity Score based on the provided parameters.

    Parameters:
    - sex (str): Gender assigned at birth, can be 'male' or 'female' (not case-sensitive).
    - age (int): Age in years.
    - respiratory_rate (int): Respiratory rate in breaths per minute.
    - oxygen_saturation (float): Oxygen saturation as a percent between 0 and 1.
    - heart_rate (int): Heart rate in beats per minute.
    - systolic_bp (int): Systolic BP in mmHg.
    - temperature (float): Temperature in degrees C.
    - alertness (str): Alertness level, can be 'Alert' or 'Confused or not alert' (not case-sensitive).
    - inspired_oxygen (str): Type of oxygen, can be 'Air' or 'Supplemental oxygen' (not case-sensitive).
    - performance_status (str): Performance status description.

    Returns:
    - float: Risk percentage between 0 and 1. Returns None for invalid inputs.

    Example:
    >>> priest("male", 45, 20, 0.95, 80, 110, 37.0, "Alert", "Air", "Unrestricted normal activity")
    0.02
    >>> priest("female", 85, 25, 0.91, 100, 85, 39.5, "Confused or not alert", "Supplemental oxygen", "Bed/chair bound, no self-care")
    0.59
    >>> priest("male", 50, 12, 0.98, 60, 120, 36.5, "Alert", "Air", "Limited strenuous activity, can do light activity")
    0.03
    """

    # Scoring for each parameter
    score = 0

    # Score based on sex
    score += 1 if sex.lower() == "male" else 0

    # Score based on age
    if 50 <= age <= 65:
        score += 2
    elif 66 <= age <= 80:
        score += 3
    elif age > 80:
        score += 4

    # Score based on respiratory rate
    if respiratory_rate < 9 or respiratory_rate > 24:
        score += 3
    elif 9 <= respiratory_rate <= 11 or 21 <= respiratory_rate <= 24:
        score += 1
    elif 12 <= respiratory_rate <= 20:
        score += 0

    # Score based on oxygen saturation
    if oxygen_saturation > 0.95:
        score += 0
    elif 0.94 <= oxygen_saturation <= 0.95:
        score += 1
    elif 0.92 <= oxygen_saturation <= 0.93:
        score += 2
    elif oxygen_saturation < 0.92:
        score += 3

    # Score based on heart rate
    if heart_rate < 41 or heart_rate > 130:
        score += 3
    elif 41 <= heart_rate <= 50 or 111 <= heart_rate <= 130:
        score += 1
    elif 51 <= heart_rate <= 90:
        score += 0
    elif 91 <= heart_rate <= 110:
        score += 1

    # Score based on systolic blood pressure
    if systolic_bp < 91 or systolic_bp > 219:
        score += 3
    elif 91 <= systolic_bp <= 100:
        score += 2
    elif 101 <= systolic_bp <= 110:
        score += 1
    elif 111 <= systolic_bp <= 219:
        score += 0

    # Score based on temperature
    if temperature < 35.1:
        score += 3
    elif 35.1 <= temperature <= 36:
        score += 1
    elif 36.1 <= temperature <= 38:
        score += 0
    elif 38.1 <= temperature <= 39:
        score += 1
    elif temperature > 39:
        score += 2

    # Score based on alertness
    if alertness.lower() == "confused or not alert":
        score += 3

    # Score based on inspired oxygen
    if inspired_oxygen.lower() == "supplemental oxygen":
        score += 2

    # Score based on performance status
    performance_status_scores = {
        "unrestricted normal activity": 0,
        "limited strenuous activity, can do light activity": 1,
        "limited activity, can self-care": 2,
        "limited self-care": 3,
        "bed/chair bound, no self-care": 4
    }
    score += performance_status_scores.get(performance_status.lower(), 0)

    # Mapping score to probability of adverse outcome using conditions
    if score in [0, 1]:
        return 0.01
    elif score in [2, 3]:
        return 0.02
    elif score == 4:
        return 0.03
    elif score == 5:
        return 0.09
    elif score == 6:
        return 0.15
    elif score == 7:
        return 0.18
    elif score == 8:
        return 0.22
    elif score == 9:
        return 0.26
    elif score == 10:
        return 0.29
    elif score == 11:
        return 0.34
    elif score == 12:
        return 0.38
    elif score == 13:
        return 0.46
    elif score == 14:
        return 0.47
    elif score == 15:
        return 0.49
    elif score == 16:
        return 0.55
    elif 17 <= score <= 25:
        return 0.59
    elif score >= 26:
        return 0.99


In [ ]:
def process_people(file_url):
    """
    Process the people from the given file URL to determine their hospital and address.

    Parameters:
    - file_url (str): The URL of the pipe-delimited file containing the people's data.

    Returns:
    - list: A list of dictionaries containing the patient's data and their hospital and address.
    """

    response = requests.get(file_url)

    # Check if the request was successful
    if response.status_code != 200:
        return None

    # Split the response text into lines
    lines = response.text.strip().split('\n')

    # Initialize the results list
    results = []

    # Process each line (person) in the file
    for line in lines[1:]:  # Skip the header line
        parts = line.split('|')
        patient_number = parts[0]
        sex = parts[1]
        age = int(parts[2])
        breath = int(parts[3])
        o2sat = float(parts[4])
        heart = int(parts[5])
        systolic = int(parts[6])
        temp = float(parts[7])
        alertness = parts[8]
        inspired = parts[9]
        status = parts[10]

        # Calculate the PRIEST score and determine the risk percentage
        risk_pct = priest(sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status)

        # Find the hospital based on the risk percentage
        hospital = find_hospital(age, sex, risk_pct)

        # Get the address of the hospital
        address = get_address(hospital)

        # Add the results to the results list
        results.append({
            patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, risk_pct, hospital, address]
        })

    return results


results = process_people("https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh")
print(results[:5])  # This will print the first 5 results for brevity.


In [ ]:
def summarize_results(patients):
    """
    Description:
    This summarize_results(pateints) takes input as patients and categorizes patients based on risk probability and provides a hospital count dictionary.

    Parameters:
    patients (dict): Patient data including risk probability and hospital information.

    Returns:
    dict: Returns a dictionary contains count of patients are now being sent to each location of hospital.
    """

    result = {}
    for key, value in patients.items():
        if value[10]<0.1:
            patients[key][11]="Home"
            patients[key][12]="Home"

    listOfHospitals = set()

    for key, value in patients.items():
        list = value[11]
        listOfHospitals.add(list)

    i = iter(listOfHospitals)
    val = next(i, None)
    while val is not None:
      result[val] = 0
      val = next(i, None)

    for val in listOfHospitals:
        for key, value in patients.items():
            if val == value[11]:
                result[val] =  result[val] + 1

    return result


summarize_results(process_people("https://hds5210-data.s3.amazonaws.com/people.psv"))